In [75]:
import pandas as pd

In [76]:

user_log = pd.read_csv('user_log_format1[161-320].csv', iterator=True, chunksize=10000)
user_log = pd.concat(user_log, ignore_index=True)

In [77]:
user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,149002,282,885,3791,1500.0,1111,0
1,149002,282,885,3791,1500.0,1107,0
2,109881,230,629,1,1662.0,820,0
3,109881,230,629,1,1662.0,820,0
4,109881,230,629,1,1662.0,820,0


In [78]:

user_info = pd.read_csv('user_info_format1.csv', iterator=True, chunksize=10000)
user_info = pd.concat(user_info, ignore_index=True)
user_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [79]:
user_log.isnull().any()

user_id        False
item_id        False
cat_id         False
seller_id      False
brand_id        True
time_stamp     False
action_type    False
dtype: bool

In [80]:
user_info.isnull().any()

user_id      False
age_range     True
gender        True
dtype: bool

In [81]:
from sklearn.preprocessing import label_binarize

In [82]:
import numpy as np

## User profile

In [83]:
user_gender = user_info[['user_id','gender']]
user_gender = user_gender.fillna(2)
user_gender.head()

,user_id,gender
0,376517,1.0
1,234512,0.0
2,344532,0.0
3,186135,0.0
4,30230,0.0


In [84]:
user_gender_info = label_binarize(np.array(user_gender.gender), classes=[0,1,2])
user_gender_info = pd.DataFrame(data=user_gender_info, columns=['gender_female','gender_male','gender_unknown'])
user_gender['user_gender_female']=user_gender_info['gender_female']
user_gender['user_gender_male']=user_gender_info['gender_male']
user_gender['user_gender_unknown']=user_gender_info['gender_unknown']
user_gender = user_gender.drop(columns = ['gender'])

In [85]:
user_gender.head()

,user_id,user_gender_female,user_gender_male,user_gender_unknown
0,376517,0,1,0
1,234512,1,0,0
2,344532,1,0,0
3,186135,1,0,0
4,30230,1,0,0


In [86]:
user_age = user_info[['user_id','age_range']].copy()
user_age = user_age.fillna(9)
user_age = user_age.replace(8,7)
user_age = user_age.drop_duplicates()
user_age_info = label_binarize(np.array(user_age.age_range),classes=[1,2,3,4,5,6,7,9])
user_age_info = pd.DataFrame(data= user_age_info, columns = ['age_0_18','age_18_24','age_25_29','age_30_34','age_35_39','age_40_49','age_50_','age_unknown'])
user_age['user_age_0_18']=user_age_info['age_0_18']
user_age['user_age_18_24']=user_age_info['age_18_24']
user_age['user_age_25_29']=user_age_info['age_25_29']
user_age['user_age_30_34']=user_age_info['age_30_34']
user_age['user_age_35_39']=user_age_info['age_35_39']
user_age['user_age_40_49']=user_age_info['age_40_49']
user_age['user_age_50_']=user_age_info['age_50_']
user_age['user_age_unknown']=user_age_info['age_unknown']
user_age = user_age.drop(columns=['age_range'])
user_age.head()

,user_id,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,user_age_40_49,user_age_50_,user_age_unknown
0,376517,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,234512,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,344532,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,186135,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,30230,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Action count

In [125]:
user_action = user_log.groupby('user_id').size().to_frame().reset_index()
user_action.columns = ['user_id','user_total_action']
user_action

,user_id,user_total_action
0,297,1
1,867,1
2,997,1
3,1076,1
4,1295,1
...,...,...
3666,423487,1
3667,423557,1
3668,423626,1
3669,423811,1


In [88]:
seller_action = user_log.groupby('seller_id').size().to_frame().reset_index()
seller_action.columns = ['seller_id','seller_total_action']
seller_action.head()

,seller_id,seller_total_action
0,1,45
1,5,2
2,25,4
3,107,1
4,139,30


In [89]:
user_seller_action = user_log.groupby(['user_id','seller_id']).size().to_frame().reset_index()
user_seller_action.columns = ['user_id','seller_id','user_seller_total_action']
user_seller_action.head()

,user_id,seller_id,user_seller_total_action
0,297,637,1
1,867,4760,1
2,997,4257,1
3,1076,3989,1
4,1295,3989,1


In [90]:
user_activity = user_log[['user_id','action_type']].copy()
user_activity_info = label_binarize(np.array(user_activity.action_type), classes=[0,1,2,3])
user_activity_info = pd.DataFrame(data = user_activity_info, columns=['Click','add_to_cart','purchase','add_to_favorite'])
user_activity['user_Click']= user_activity_info.Click
user_activity['user_Add_to_cart']=user_activity_info.add_to_cart
user_activity['user_purchase']=user_activity_info.purchase
user_activity['user_Add_to_favorite']=user_activity_info.add_to_favorite
user_activity_count = user_activity.groupby('user_id').agg('sum').reset_index()
user_activity_count.columns = ['user_id','action_type','user_Click_Count','user_Add_to_cart_count','user_purchase_count','user_Add_to_favorite_count']
user_activity_count = user_activity_count.drop(columns=['action_type'])
user_activity_count.head()

,user_id,user_Click_Count,user_Add_to_cart_count,user_purchase_count,user_Add_to_favorite_count
0,297,1,0,0,0
1,867,1,0,0,0
2,997,1,0,0,0
3,1076,1,0,0,0
4,1295,1,0,0,0


In [91]:
seller_activity = user_log[['seller_id','action_type']].copy()
seller_activity_info = label_binarize(np.array(seller_activity.action_type), classes=[0,1,2,3])
seller_activity_info = pd.DataFrame(data = seller_activity_info, columns=['Click','add_to_cart','purchase','add_to_favorite'])
seller_activity['seller_Click']= seller_activity_info.Click
seller_activity['seller_Add_to_cart']=seller_activity_info.add_to_cart
seller_activity['seller_purchase']=seller_activity_info.purchase
seller_activity['seller_Add_to_favorite']=seller_activity_info.add_to_favorite
seller_activity_count = seller_activity.groupby('seller_id').agg('sum').reset_index()
seller_activity_count.columns = ['seller_id','action_type','seller_Click_Count','seller_Add_to_cart_count','seller_purchase_count','seller_Add_to_favorite_count']
seller_activity_count = seller_activity_count.drop(columns=['action_type'])
seller_activity_count.head()

,seller_id,seller_Click_Count,seller_Add_to_cart_count,seller_purchase_count,seller_Add_to_favorite_count
0,1,43,0,0,2
1,5,1,0,0,1
2,25,4,0,0,0
3,107,1,0,0,0
4,139,28,0,0,2


In [92]:
user_seller_item = user_log[['user_id','seller_id','action_type']].copy()
user_seller_item_info = label_binarize(np.array(user_seller_item.action_type),classes=[0,1,2,3])
user_seller_item_info = pd.DataFrame(data= user_seller_item_info, columns=['Click','Add_to_cart','Purchase','Add_to_favorite'])
user_seller_item['user_seller_Click_count']= user_seller_item_info.Click
user_seller_item['user_seller_Add_to_cart_count']=user_seller_item_info.Add_to_cart
user_seller_item['user_seller_Purchase_count']=user_seller_item_info.Purchase
user_seller_item['user_seller_Add_to_favorite_count']=user_seller_item_info.Add_to_favorite
user_seller_item_act_count = user_seller_item.groupby(['user_id','seller_id']).agg('sum').reset_index()
user_seller_item_act_count = user_seller_item_act_count.drop(columns = ['action_type'])
user_seller_item_act_count.head()

,user_id,seller_id,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count
0,297,637,1,0,0,0
1,867,4760,1,0,0,0
2,997,4257,1,0,0,0
3,1076,3989,1,0,0,0
4,1295,3989,1,0,0,0


## Day count

In [93]:
user_active_days = user_log[['user_id','time_stamp']].drop_duplicates()
user_active_days_info = user_active_days.groupby('user_id').size().reset_index()
user_active_days_info.columns = ['user_id','user_active_days']
user_active_days_info.head()

,user_id,user_active_days
0,297,1
1,867,1
2,997,1
3,1076,1
4,1295,1


In [94]:
seller_active_days = user_log[['seller_id','time_stamp']].drop_duplicates()
seller_active_days_info = seller_active_days.groupby('seller_id').size().reset_index()
seller_active_days_info.columns = ['seller_id','seller_active_days']
seller_active_days_info.head()

,seller_id,seller_active_days
0,1,36
1,5,2
2,25,4
3,107,1
4,139,19


In [95]:
user_seller_days = user_log[['user_id','seller_id','time_stamp']].copy()
user_seller_days.drop_duplicates(inplace=True)
user_seller_days = user_seller_days.groupby(['user_id','seller_id']).size().reset_index()
user_seller_days.columns = ['user_id','seller_id','user_seller_active_days']
user_seller_days

,user_id,seller_id,user_seller_active_days
0,297,637,1
1,867,4760,1
2,997,4257,1
3,1076,3989,1
4,1295,3989,1
...,...,...,...
3702,423487,2340,1
3703,423557,2871,1
3704,423626,145,1
3705,423811,3323,1


## Product diversity

In [96]:
seller_item = user_log[['seller_id','item_id']].copy()
seller_item.drop_duplicates(['item_id'], inplace=True)
seller_item_info = seller_item.groupby('seller_id').size().reset_index()
seller_item_info.columns = ['seller_id','seller_item_count']
seller_item_info.head()                                     

,seller_id,seller_item_count
0,1,1
1,5,1
2,25,1
3,107,1
4,139,1


In [97]:
seller_brand = user_log[['seller_id','brand_id']].copy()
seller_brand.drop_duplicates(['brand_id'], inplace=True)
seller_brand_info = seller_brand.groupby('seller_id').size().reset_index()
seller_brand_info.columns = ['seller_id','seller_brand_count']
seller_brand_info.head()

,seller_id,seller_brand_count
0,1,1
1,5,1
2,25,1
3,107,1
4,139,1


In [98]:
seller_cat = user_log[['seller_id','cat_id']].copy()
seller_cat.drop_duplicates(['cat_id'], inplace=True)
seller_cat_info = seller_cat.groupby('seller_id').size().reset_index()
seller_cat_info.columns = ['seller_id','seller_category_count']
seller_cat_info.head()

,seller_id,seller_category_count
0,1,1
1,5,1
2,107,1
3,158,1
4,246,1


In [99]:
user_item = user_log[['user_id','item_id']].copy()
user_item.drop_duplicates(['item_id'],inplace=True)
user_item_info = user_item.groupby('user_id').size().reset_index()
user_item_info.columns = ['user_id','user_item_count']
user_item_info.head()

,user_id,user_item_count
0,5857,1
1,7692,1
2,12778,1
3,13202,1
4,14540,1


In [100]:
user_cat = user_log[['user_id','cat_id']].copy()
user_cat.drop_duplicates(['cat_id'],inplace=True)
user_cat_info = user_cat.groupby('user_id').size().reset_index()
user_cat_info.columns = ['user_id','user_category_count']
user_cat_info.head()

,user_id,user_category_count
0,7692,1
1,12778,1
2,14540,1
3,14747,1
4,15445,1


In [101]:
user_brand = user_log[['user_id','brand_id']].copy()
user_brand.drop_duplicates(['brand_id'],inplace=True)
user_brand_info = user_cat.groupby('user_id').size().reset_index()
user_brand_info.columns = ['user_id','user_brand_count']
user_brand_info.head()

,user_id,user_brand_count
0,7692,1
1,12778,1
2,14540,1
3,14747,1
4,15445,1


In [102]:
brand_cat = user_log[['brand_id','cat_id']].copy()
brand_cat.drop_duplicates(['brand_id','cat_id'], inplace=True)
brand_cat_info = brand_cat.groupby('brand_id').size().reset_index()
brand_cat_info.columns = ['brand_id','brand_category_count']
brand_cat_info.head()

,brand_id,brand_category_count
0,68.0,1
1,69.0,1
2,99.0,1
3,136.0,1
4,178.0,1


In [103]:
cat_item = user_log[['cat_id','item_id']].copy()
cat_item.drop_duplicates(['cat_id','item_id'], inplace=True)
cat_item_info = cat_item.groupby('cat_id').size().reset_index()
cat_item_info.columns = ['cat_id','category_item_count']
cat_item_info.head()

,cat_id,category_item_count
0,35,2
1,71,1
2,103,1
3,108,1
4,115,1


In [104]:
user_seller_item = user_log[['user_id','seller_id','item_id']].copy()
user_seller_item.drop_duplicates(inplace=True)
user_seller_item = user_seller_item.groupby(['user_id','seller_id']).size().reset_index()
user_seller_item.columns = ['user_id','seller_id','user_seller_item_count']
user_seller_item.head()

,user_id,seller_id,user_seller_item_count
0,297,637,1
1,867,4760,1
2,997,4257,1
3,1076,3989,1
4,1295,3989,1


In [105]:
user_seller_brand = user_log[['user_id','seller_id','brand_id']].copy()
user_seller_brand.drop_duplicates(inplace=True)
user_seller_brand = user_seller_brand.groupby(['user_id','seller_id']).size().reset_index()
user_seller_brand.columns = ['user_id','seller_id','user_seller_brand_count']
user_seller_brand.head()

,user_id,seller_id,user_seller_brand_count
0,297,637,1
1,867,4760,1
2,997,4257,1
3,1076,3989,1
4,1295,3989,1


In [106]:
user_seller_cat = user_log[['user_id','seller_id','cat_id']].copy()
user_seller_cat.drop_duplicates(inplace=True)
user_seller_cat = user_seller_cat.groupby(['user_id','seller_id']).size().reset_index()
user_seller_cat.columns = ['user_id','seller_id','user_seller_category_count']
user_seller_cat.head()

,user_id,seller_id,user_seller_category_count
0,297,637,1
1,867,4760,1
2,997,4257,1
3,1076,3989,1
4,1295,3989,1


## User-merchant similarity

In [107]:
seller_mkt_share_brand = user_log[['seller_id','brand_id']].copy()
seller_mkt_share_brand = seller_mkt_share_brand.groupby(['seller_id','brand_id']).size().reset_index()
seller_mkt_share_brand 

,seller_id,brand_id,0
0,1,1662.0,45
1,5,7529.0,2
2,25,5809.0,4
3,107,4509.0,1
4,139,3228.0,30
...,...,...,...
150,4882,5230.0,23
151,4955,7924.0,58
152,4957,1567.0,3
153,4964,5506.0,1


In [108]:
# merchant’s market share on the brand = NMB/NB
seller_mkt_share_brand = user_log[['seller_id','brand_id']].copy()
seller_mkt_share_brand = seller_mkt_share_brand.groupby(['seller_id','brand_id']).size().reset_index()
seller_mkt_share_brand.columns = ['seller_id','brand_id','NMB']
seller_mkt_share_brand['seller_mkt_share_brand'] = 100*(seller_mkt_share_brand['NMB']/seller_mkt_share_brand['NMB'].sum())
seller_mkt_share_brand = seller_mkt_share_brand.drop(columns=['NMB'])
#seller_mkt_share_brand['seller_mkt_share_brand'].sum()

Similarity scores are calculated based on the merchant’s market share features and the preferences of users on brands/categories 

```user’s preference on the brand/category = # of actions on the brand/category```

```user-merchant similarity score = merchant’s market share × user’s preference```

E.g. a merchant has 5 brands with respective market shares ```(0.1, 0.2, 0.05, 0.3, 0.01)```, and the number of times of a user buying the 5 brands are ```(0, 1, 2, 0, 2)```, then the brand similarity score is the sum of inner product of merchant’s market share and user’s preference:``` 0.1 × 0 + 0.2 × 1 + 0.05 × 2 + 0.3 × 0 + 0.01 × 2 = 0.32```


In [109]:
# user-merchant similarity on brand
user_mkt_share_brand = user_log.groupby(['user_id','brand_id']).size().reset_index()
user_mkt_share_brand.columns = ['user_id','brand_id','NUB']
user_seller_brand_similarity = user_log[['user_id','seller_id','brand_id']].copy()
user_seller_brand_similarity.drop_duplicates(inplace=True)
user_seller_brand_similarity = pd.merge(user_seller_brand_similarity,seller_mkt_share_brand,on=['seller_id','brand_id'],how='left')
user_seller_brand_similarity = pd.merge(user_seller_brand_similarity,user_mkt_share_brand,on=['user_id','brand_id'],how='left')
user_seller_brand_similarity.fillna(0)
user_seller_brand_similarity['brand_similarity_score'] = user_seller_brand_similarity['seller_mkt_share_brand']*user_seller_brand_similarity['NUB']
user_seller_brand_similarity = user_seller_brand_similarity.groupby(['user_id','seller_id'])['brand_similarity_score'].agg('sum').reset_index()
user_seller_brand_similarity.head()

,user_id,seller_id,brand_similarity_score
0,297,637,13.519707
1,867,4760,6.080049
2,997,4257,4.689887
3,1076,3989,20.394134
4,1295,3989,20.394134


In [110]:
# user-merchant similarity on category
seller_mkt_share_cat = user_log[['seller_id','cat_id']].copy()
seller_mkt_share_cat = seller_mkt_share_cat.groupby(['seller_id','cat_id']).size().reset_index()
seller_mkt_share_cat.columns = ['seller_id','cat_id','NMC']
seller_mkt_share_cat['seller_mkt_share_cat'] = seller_mkt_share_cat['NMC']/seller_mkt_share_cat.groupby('cat_id')['NMC'].transform('sum')
seller_mkt_share_cat = seller_mkt_share_cat.drop(columns=['NMC'])

user_mkt_share_cat = user_log.groupby(['user_id','cat_id']).size().reset_index()
user_mkt_share_cat.columns = ['user_id','cat_id','NUC']

user_seller_cat_similarity = user_log[['user_id','seller_id','cat_id']].copy()
user_seller_cat_similarity.drop_duplicates(inplace=True)
user_seller_cat_similarity = pd.merge(user_seller_cat_similarity,seller_mkt_share_cat,on=['seller_id','cat_id'],how='left')
user_seller_cat_similarity = pd.merge(user_seller_cat_similarity,user_mkt_share_cat,on=['user_id','cat_id'],how='left')
user_seller_cat_similarity.fillna(0)
user_seller_cat_similarity['cat_similarity_score'] = user_seller_cat_similarity['seller_mkt_share_cat']*user_seller_cat_similarity['NUC']
user_seller_cat_similarity = user_seller_cat_similarity.groupby(['user_id','seller_id'])['cat_similarity_score'].agg('sum').reset_index()
user_seller_cat_similarity.head()

,user_id,seller_id,cat_similarity_score
0,297,637,0.938494
1,867,4760,0.626772
2,997,4257,0.933131
3,1076,3989,1.000000
4,1295,3989,1.000000


In [111]:
# user-level feature

# demographic
user_level_feature = pd.merge(user_gender, user_age, on=['user_id'],how='left')

# counts
# day counts
user_level_feature = pd.merge(user_level_feature, user_active_days_info, on=['user_id'],how='left')
# action counts
user_level_feature = pd.merge(user_level_feature, user_action, on=['user_id'],how='left')
user_level_feature = pd.merge(user_level_feature, user_activity_count, on=['user_id'],how='left')

# product diversity
user_level_feature = pd.merge(user_level_feature, user_item_info, on=['user_id'],how='left')
user_level_feature = pd.merge(user_level_feature, user_cat_info, on=['user_id'],how='left')
user_level_feature = pd.merge(user_level_feature, user_brand_info, on=['user_id'],how='left')

user_level_feature.head()

,user_id,user_gender_female,user_gender_male,user_gender_unknown,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,user_age_40_49,...,user_age_unknown,user_active_days,user_total_action,user_Click_Count,user_Add_to_cart_count,user_purchase_count,user_Add_to_favorite_count,user_item_count,user_category_count,user_brand_count
0,376517,0,1,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,234512,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,344532,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,186135,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30230,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
user_level_feature.shape

(424170, 21)

In [113]:
# seller-level feature

# product diversity
seller_level_feature = pd.merge(seller_item_info, seller_cat_info, on=['seller_id'],how='left')
seller_level_feature = pd.merge(seller_level_feature, seller_brand_info, on=['seller_id'],how='left')

# counts
# day counts
seller_level_feature = pd.merge(seller_level_feature, seller_active_days_info, on=['seller_id'],how='left')
# action counts
seller_level_feature = pd.merge(seller_level_feature, seller_action, on=['seller_id'],how='left')
seller_level_feature = pd.merge(seller_level_feature, seller_activity_count, on=['seller_id'],how='left')

seller_level_feature.head()

,seller_id,seller_item_count,seller_category_count,seller_brand_count,seller_active_days,seller_total_action,seller_Click_Count,seller_Add_to_cart_count,seller_purchase_count,seller_Add_to_favorite_count
0,1,1,1.0,1.0,36,45,43,0,0,2
1,5,1,1.0,1.0,2,2,1,0,0,1
2,25,1,NaN,1.0,4,4,4,0,0,0
3,107,1,1.0,1.0,1,1,1,0,0,0
4,139,1,NaN,1.0,19,30,28,0,0,2


In [114]:
seller_level_feature.shape

(149, 10)

In [115]:
# user-seller interaction level feature

# product diversity
user_seller_level_feature = pd.merge(user_seller_item, user_seller_cat, on=['user_id','seller_id'],how='left')
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_brand, on=['user_id','seller_id'],how='left')

# counts
# action counts
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_item_act_count, on=['user_id','seller_id'],how='left')
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_action, on=['user_id','seller_id'],how='left')
# day counts
user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_days,on=['user_id','seller_id'],how='left') 

# similarity
user_seller_level_feature = pd.merge(user_seller_level_feature,user_seller_brand_similarity,on=['user_id','seller_id'],how='left') 
user_seller_level_feature = pd.merge(user_seller_level_feature,user_seller_cat_similarity,on=['user_id','seller_id'],how='left')


user_seller_level_feature.head()

,user_id,seller_id,user_seller_item_count,user_seller_category_count,user_seller_brand_count,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count,user_seller_total_action,user_seller_active_days,brand_similarity_score,cat_similarity_score
0,297,637,1,1,1,1,0,0,0,1,1,13.519707,0.938494
1,867,4760,1,1,1,1,0,0,0,1,1,6.080049,0.626772
2,997,4257,1,1,1,1,0,0,0,1,1,4.689887,0.933131
3,1076,3989,1,1,1,1,0,0,0,1,1,20.394134,1.000000
4,1295,3989,1,1,1,1,0,0,0,1,1,20.394134,1.000000


In [116]:
user_seller_level_feature.shape

(3707, 13)

In [117]:

train_df = pd.read_csv('train_format1.csv')
train_df.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [118]:

test_df = pd.read_csv('test_format1.csv')
test_df.head()

,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN
2,98688,1964,NaN
3,98688,3645,NaN
4,295296,3361,NaN


In [119]:
#combine train data and new added feature
#add user-level feature
train_complete = pd.merge(train_df, user_level_feature, on=['user_id'],how='left')
#add seller-level feature
seller_level_feature = seller_level_feature.rename(columns = {'seller_id':'merchant_id'})
train_complete = pd.merge(train_complete, seller_level_feature, on=['merchant_id'],how='left')
#add user-seller level feature
user_seller_level_feature = user_seller_level_feature.rename(columns = {'seller_id':'merchant_id'})
train_complete = pd.merge(train_complete, user_seller_level_feature, on=['user_id','merchant_id'],how='left')
train_complete.head()

,user_id,merchant_id,label,user_gender_female,user_gender_male,user_gender_unknown,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,...,user_seller_category_count,user_seller_brand_count,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count,user_seller_total_action,user_seller_active_days,brand_similarity_score,cat_similarity_score
0,34176,3906,0,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34176,121,0,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34176,4356,1,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34176,2217,0,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,230784,4818,0,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
train_complete.to_csv (r'train_complete.csv', index = False, header=True)

In [121]:
train_complete.shape

(260864, 43)

In [122]:
#combine test data and new added feature
#add user-level feature
test_complete = pd.merge(test_df, user_level_feature, on=['user_id'],how='left')
#add seller-level feature
test_complete = pd.merge(test_complete, seller_level_feature,on=['merchant_id'],how='left')
#add user-seller-level feature
test_complete = pd.merge(test_complete, user_seller_level_feature, on=['user_id','merchant_id'],how='left')
test_complete.head()


,user_id,merchant_id,prob,user_gender_female,user_gender_male,user_gender_unknown,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,...,user_seller_category_count,user_seller_brand_count,user_seller_Click_count,user_seller_Add_to_cart_count,user_seller_Purchase_count,user_seller_Add_to_favorite_count,user_seller_total_action,user_seller_active_days,brand_similarity_score,cat_similarity_score
0,163968,4605,NaN,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,360576,1581,NaN,0,0,1,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,98688,1964,NaN,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,98688,3645,NaN,1,0,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,295296,3361,NaN,0,1,0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
test_complete.shape

(261477, 43)

In [124]:
test_complete.to_csv (r'test_complete.csv', index = False, header=True)